In [5]:
from google.cloud import storage
import os
import netCDF4
import numpy as np
import numpy.ma as ma
from dask import delayed
import dask
import dask.array as da
from dask.distributed import Client
import glob
import tempfile
import subprocess
import datetime, time
from urllib import request
from multiprocessing import Pool
import json
import gc
import boto3
import botocore
import itertools

storage_client = storage.Client.from_service_account_json('/home/jovyan/work/credentials.json')
bucket = storage_client.get_bucket('nex-gddp')

loca_bucket = 'nasanex'
base_key_path = 'LOCA'
base_url = 'ftp://gdo-dcp.ucllnl.org/pub/dcp/archive/cmip5/loca/LOCA_2016-04-02/'
all_models = ["ACCESS1-0","ACCESS1-3","CCSM4","CESM1-BGC","CESM1-CAM5","CMCC-CM","CMCC-CMS","CNRM-CM5","CSIRO-Mk3-6-0","CanESM2","EC-EARTH","FGOALS-g2","GFDL-CM3","GFDL-ESM2G","GFDL-ESM2M","GISS-E2-H","GISS-E2-R","HadGEM2-AO","HadGEM2-CC","HadGEM2-ES","IPSL-CM5A-LR","IPSL-CM5A-MR","MIROC-ESM","MIROC-ESM-CHEM","MIROC5","MPI-ESM-LR","MPI-ESM-MR","MRI-CGCM3","NorESM1-M","bcc-csm1-1","bcc-csm1-1-m","inmcm4"]
some_models = ["ACCESS1-0","ACCESS1-3","CCSM4"]

#client = Client('scheduler:8786')
dask.set_options(get=dask.get)


s3 = boto3.resource('s3')


# Begin here
def process_model_year(model, scenario, year):
    temps = process_year_temps(model, scenario, year)
    return temps

def process_year_temps(model, scenario, year):
    print(f"Processing temperatures for {model} {year} ({scenario})")
    ids = (gen_netcdf_id(model, scenario, year, 'tasmax'), gen_netcdf_id(model, scenario, year, 'tasmin'))
    print(f"File ids are: {ids}")
    tasmax_file, tasmin_file = list(map(download_file, ids))
    tasmax_dataset, tasmin_dataset = netCDF4.Dataset(tasmax_file), netCDF4.Dataset(tasmin_file)
    tasmax_arr = da.from_array(tasmax_dataset['tasmax'], chunks = (365, 490, 960))
    tasmin_arr = da.from_array(tasmin_dataset['tasmin'], chunks = (365, 490, 960))
    tasavg_arr = np.mean(da.stack((tasmax_arr, tasmin_arr)), axis = 0)
    
    # Annual means
    print("Calc avg_tasmin")
    avg_tasmin = np.mean(tasmin_arr, axis = 0).compute()
    print(avg_tasmin)
    print("Calc avg_tasmax")
    avg_tasmax = np.mean(tasmax_arr, axis = 0).compute()
    print(avg_tasmax)
    print("Calc avg_tasavg")
    avg_tasavg = np.mean(tasavg_arr, axis = 0).compute()
    print(avg_tasavg)
    
    hdds = delayed(hdd)(tasavg_arr, axis = 0).compute()
    cdds = delayed(cdd)(tasavg_arr, axis = 0).compute()
    ffs = delayed(frost_free_season)(tasmin_arr, axis = 0).compute()
    
    res = np.stack((
        avg_tasmax,
        avg_tasmin,
        avg_tasavg,
        hdds,
        cdds,
        ffs
    ))
    
    output_filename = f'{year}_{model}_processed_temperatures.npy'
    np.save('/temp/' + output_filename, results)
    blob = bucket.blob(output_filename)
    blob.upload_from_filename('/temp/' + output_filename)
    os.remove('/temp/' + output_filename)
    
    return res

def gen_netcdf_id(model, scenario, year, var):
    id = f'LOCA/{model}/16th/{scenario}/r1i1p1/{var}/{var}_day_{model}_{scenario}_r1i1p1_{str(year)}0101-{str(year)}1231.LOCA_2016-04-02.16th.nc'
    return id

def download_file(file_id, loca_bucket = loca_bucket, download_location = '/temp'):
    filename = f'{download_location}/{file_id.split("/")[-1]}'
    print(f"Downloading {filename}")
    try:
        s3.Bucket(loca_bucket).download_file(file_id, filename)
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            file_id = file_id.replace('r1i1p1', 'r6i1p1')
            s3.Bucket(loca_bucket).download_file(file_id, filename)
    except:
        filename = None
    return filename

def cleanup():
    for file in glob.glob('/temp/*'):
        os.remove(file)

# Actual processing
def hdd(a, axis):
    a_to_baseline = 291.483 - a
    masked = ma.masked_where(a_to_baseline <= 0, a_to_baseline)
    intermediate_matrix = ma.filled(masked, fill_value = 0)
    result = np.sum(intermediate_matrix, axis = 0)
    return result

# Cooling degree days
def cdd(a, axis):
    a_to_baseline = 291.483 - a
    a_to_baseline[a_to_baseline < -10000] = 0
    masked = ma.masked_where(a_to_baseline >= 0, a_to_baseline)
    intermediate_matrix = ma.filled(masked, fill_value = 0)
    result = np.sum(np.abs(intermediate_matrix), axis = 0)
    return result

def longest_streak(diff):
    result = 0
    try:
        result =  np.amax(
            np.array(np.where(diff < 0)) - np.array(np.where(diff > 0))
        )
    except ValueError:
        #raised if empty
        result = 0
    return result

# Longest streak of days over freezing temperature (tasmin)
def frost_free_season(a, axis):
    # First, dealing with the first matrix
    frost_days_matrix = (a > 273.15) * 1
    # We pad it with zeroes at the ends of the designed axis
    zeros_shape = list(a.shape)
    del zeros_shape[axis]
    zeros_matrix = np.expand_dims(np.zeros(zeros_shape), axis = axis)
    concat_matrix = np.concatenate((zeros_matrix, frost_days_matrix, zeros_matrix))
    # We calculate the deltas along an axis
    diff = np.diff(concat_matrix, axis = axis)
    # And get the longest streak from there --
    # apply along axis is far from ideal, but
    # np.where doesn't operate over axes, so we have to iterate
    result = np.apply_along_axis(longest_streak, axis, diff)
    return result



In [7]:
res = process_year_temps("ACCESS1-0", "historical", 1971)

Processing temperatures for ACCESS1-0 1971 (historical)
File ids are: ('LOCA/ACCESS1-0/16th/historical/r1i1p1/tasmax/tasmax_day_ACCESS1-0_historical_r1i1p1_19710101-19711231.LOCA_2016-04-02.16th.nc', 'LOCA/ACCESS1-0/16th/historical/r1i1p1/tasmin/tasmin_day_ACCESS1-0_historical_r1i1p1_19710101-19711231.LOCA_2016-04-02.16th.nc')
Calc avg_tasmin
[[  1.00000085e+30   1.00000085e+30   1.00000085e+30 ...,   1.00000085e+30
    1.00000085e+30   1.00000085e+30]
 [  1.00000085e+30   1.00000085e+30   1.00000085e+30 ...,   1.00000085e+30
    1.00000085e+30   1.00000085e+30]
 [  1.00000085e+30   1.00000085e+30   1.00000085e+30 ...,   1.00000085e+30
    1.00000085e+30   1.00000085e+30]
 ..., 
 [  1.00000085e+30   1.00000085e+30   1.00000085e+30 ...,   1.00000085e+30
    1.00000085e+30   1.00000085e+30]
 [  1.00000085e+30   1.00000085e+30   1.00000085e+30 ...,   1.00000085e+30
    1.00000085e+30   1.00000085e+30]
 [  1.00000085e+30   1.00000085e+30   1.00000085e+30 ...,   1.00000085e+30
    1.0000008

In [21]:
%matplotlib inline

res[res > 100000] = 0

import matplotlib.pyplot as plt
plt.imshow(res[6, :, :], origin = 'lower')
plt.colorbar()

IndexError: index 6 is out of bounds for axis 0 with size 6

In [ ]:
# GET /process/:year/:model/:scenario
req = json.loads(REQUEST)
year = int(req['path']['year'])
model = req['path']['model']
scenario = req['path']['scenario']
cleanup()    
result = download_and_process(model, scenario, year)
print({'output': result})